# Making Plots from Original images

In [1]:
#### Conda environment
# conda create --name terada2019 python=3.7
# pip install --upgrade pip
# pip install astropy scipy
# pip install photutils
# pip install jupyter matplotlib h5py aplpy pyregion PyAVM healpy

In [2]:
import os
import sys
import time
sys.path
sys.path.append('./')

import numpy as np
from astropy.stats import mad_std

from photutils import datasets
from photutils import DAOStarFinder
from photutils import aperture_photometry, CircularAperture

import aplpy
from astropy.io.fits import getdata
from astropy import wcs
from astropy.io import fits
from astropy import units as u
from astropy import constants as con
from astropy.coordinates import SkyCoord
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

import matplotlib
matplotlib.use('PDF')
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
# %matplotlib inline

#### Format of marker file:

name, ra_h ra_m ra_s dec_d dec_m dec_s  R:0-1 G:0-1 B:0-1 alpha  size

dm_tau  04 33 48.7335659850  +18 10 09.974471722  1 0.2 0.2 1.0 2.0

## Defining a aperture photometry pipeline class

In [5]:
class marker:
    '''
    Class for marker objects
    '''
    
    def __init__(self, label='unknown_marker'):
        self.label = label
        self.ra  = 0.0
        self.dec  = 0.0
        self.color = (0,0,0)
        self.alpha = 1.0
        self.size  = 1.0
    
    def __del__(self):
        pass
        

class apt_pipe:
    '''
    Class for apeture photometry pipeline.
    
    History:
    1. Basic version developed.  (Baobab Liu, 2019, Dec.07)
    
    2 Required Conda enviornment:
    # conda create --name terada2019 python=3.7
    # pip install --upgrade pip
    # pip install astropy scipy
    # pip install photutils
    # pip install jupyter matplotlib h5py aplpy pyregion PyAVM healpy
    
    '''

    
    def __init__(self, data_path = './',
                 markerfile   = './markers.txt',
                 ascii_report = './ascii_report.txt',
                 verbose      = False
                ):
        '''
        Initializer of the class.
        It first load image names from the FITS image directory.
        Then it checks the integrity of the FITS image headers,
        and dump report to an ASCII file.
        Finally, it loads the coordinate of the markerfile if any.
        
        Keywords:
        
        data_path [string] or list of [string] : Default: './'.
                  Can either load data from one specified directory or a list of 
                  specified directory.
        
        markerfile [string]   : File name of one or a list of markers.
                  Format :
                  name, ra_h ra_m ra_s dec_d dec_m dec_s R:0-1 G:0-1 B:0-1 alpha size
                  Example :
                  dm_tau  04 33 48.7335659850 +18 10 09.974471722   1 0.2 0.2 1.0 2.0
                  test    04 33 48.7335659850  +18 10 49.974471722  0.0 1.0 0 1.0 30.0
        
        ascii_report [string] :
        
        verbose   [True]      : Verbosely dump the status of the data.
        
        '''
                                        
        self.data_path  = data_path
        self.markerfile = markerfile
        self.verbose    = verbose
        self.num_images = 0
        self.num_markers = 0
        
        # initialize variables
        os.system('rm -rf ' + ascii_report)
        F_report = open("ascii_report.txt","w+")
        self.data_path_list = []
        self.num_data_path  = 0
        self.filterlist     = []
        self.images         = []
        self.path_dict      = {}
        self.band_dict      = {}
        self.date_dict      = {}
        self.jd_dict        = {}
        self.marker_list    = []
        
        # load image names
        if ( type(data_path) == str ):
            self.data_path_list.append(data_path)
            self.num_data_path = 1
            if (verbose==True):
                print('Loading FITS image from single directory : ' + data_path)
                
        if ( type(data_path) == list ):
            self.data_path_list.extend(data_path)
            self.num_data_path = len( self.data_path_list )
            if (verbose==True):
                for data_idx in range(0, len(data_path) ):
                    print('Loading FITS image from : ' + data_path[data_idx])
                            
        try:
            for data_idx in range(0, self.num_data_path):
                self.images.extend(  os.listdir( self.data_path_list[data_idx] )  )
                temp_names = os.listdir( self.data_path_list[data_idx] )
                # self.num_images = self.num_images + len( temp_names )
                for name in temp_names:
                    self.path_dict[name] = self.data_path_list[data_idx]
            self.num_images = len( self.images )
                
            if ( verbose == True ):
                print('##############################################################')
                print('Processing ' + str(self.num_images).strip() + ' images \n')
                print('##############################################################')
        except:
            print('No image found')
            
            
        # checking integrity of FITS headers
        F_report.write('FITS header integrity: \n')
        for i in range(0, self.num_images):
            
            image_name = self.images[i]
            hdulist = fits.open( self.path_dict[image_name] + '/' + image_name)
            try:
                crval1 = hdulist[0].header['crval1']
                crval2 = hdulist[0].header['crval2']
            except:
                if (verbose == True ):
                    print('Warning, coordinate header of ' + image_name + ' does not exist.')
                F_report.write( image_name + ' has no coordinate header \n' )
                continue
                
            try:
                date   = hdulist[0].header['date-obs']
                jd     = hdulist[0].header['jd']
                self.date_dict[image_name] = date
                self.jd_dict[image_name]  = jd
            except:
                if (verbose == True ):
                    print('Warning. Observing date of ' + image_name + ' is not known.')
                F_report.write( image_name + ' has no observing time information \n' )
                continue
                
            try:
                band   = hdulist[0].header['filter']
                if (band not in self.filterlist):
                    self.filterlist.append(band)
                self.band_dict[image_name] = band
            except:
                if (verbose == True ):
                    print('Warning. Filter name of ' + image_name + ' does not exist.')
                F_report.write( image_name + ' has unknown filter band. \n' )
                continue
                
        F_report.write('\n')
        
        # load markers if there is
        try:
            marker_label     = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=0, dtype=np.str)
            rah              = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=1 )
            ram              = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=2 )
            ras              = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=3 )
            decd             = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=4 )
            decm             = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=5 )
            decs             = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=6 )
            markerR          = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=7 )
            markerG          = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=8 )
            markerB          = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=9 )
            marker_alpha     = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=10)
            marker_size      = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=11)
                        
            ra  = ( rah + ram / 60.0 + ras / 3600.0 ) * 15.0
            dec_sign = np.where( (decd>0), 1.0, -1.0)
            dec      = decd + dec_sign * decm / 60.0 + dec_sign * decs / 3600.0
                
            self.num_markers = np.size(ra)
            if (verbose == True):
                print('Number of markers : ' + str(self.num_markers).strip() )
                
            # loading to marker objects
            for mid in range(0, self.num_markers):
                if (self.num_markers > 1):
                    temp_marker       = marker(label=marker_label[mid])
                    temp_marker.ra    = ra[mid] 
                    temp_marker.dec   = dec[mid]
                    temp_marker.color = (markerR[mid], markerG[mid], markerB[mid])
                    temp_marker.alpha = (marker_alpha[mid])
                    temp_marker.size  = (marker_size[mid])
                    self.marker_list.append(temp_marker)
                else:
                    temp_marker       = marker(label=marker_label)
                    temp_marker.ra    = ra
                    temp_marker.dec   = dec
                    temp_marker.color = (markerR, markerG, markerB)
                    temp_marker.alpha = (marker_alpha)
                    temp_marker.size  = (marker_size)
                    self.marker_list.append(temp_marker)
            
        except:
            if (verbose == True):
                print('No markers loaded')

        # Dumping more information ot the report
        # filters
        F_report.write('Used filters: \n')
        outstring = ' '
        for tempstr in self.filterlist:
            outstring = outstring + tempstr + ' '
        F_report.write(outstring + '\n')
        
        F_report.close()

        
        
    def __del__(self):
        pass
    
    
    
    def plot_preview(self, output_directory='./preview_images',
                     label_marker=True,
                     fig_format='png'
                    ):
        '''
        Function to plot the preview figures for FITS imagfes.
        It will produce figures for the observed images (one sub-directory for each band),
        and collect them into an output_directory.
        
        
        Keywords:
        
        output_directory [string] : The directory to collect the output images.
        
        label_marker [True/False] : If True, label the names of the markers on the figure.
        
        fig_format   [string]     : 'png' or 'pdf'
        '''
        
        F_report = open("ascii_report.txt","a+")
        F_report.write('\n')
        F_report.write('Image preview summary: \n')
        
        os.system('rm -rf ' + output_directory)
        os.system('mkdir '  + output_directory)
        
        output_path_dict = {}
        for tempstr in self.filterlist:
            output_path_dict[tempstr] = './' + tempstr + 'band_maps'
            
        for key in output_path_dict.keys():
            os.system('rm -rf ' + output_path_dict[key] )
            os.system('mkdir '  + output_path_dict[key] )
            
        # plot images
        for i in range(0, self.num_images):
            image_name = self.images[i]
            try:
                band       = self.band_dict[image_name]
                jd         = self.jd_dict[image_name]
            except:
                F_report.write('Warning. Image ' +  image_name + ' is not saved. \n')
                continue
            
            # plot image
            fig = aplpy.FITSFigure( self.path_dict[image_name] + '/' + image_name)
            fig.show_grayscale(invert=False)
            
            # plot symbol
            
            # define output figure name
            try:
                outfig_name = \
                              band + '_' + \
                              str( round(jd,4) ) + '.' + fig_format
                # fig.set_xaxis_coord_type('longitude')
                # fig.set_yaxis_coord_type('latitude')
                fig.axis_labels.hide()
                fig.show_grayscale(invert=False)
            except:
                if (self.verbose == True):
                    F_report.write('Warning. Image ' +  image_name + ' is not saved. \n')
                    
            # plot markers
            for marker in self.marker_list:
                # plot markers in the png figure
                fig.show_markers(
                                 marker.ra, marker.dec, 
                                 edgecolor=marker.color, 
                                 # facecolor=facecolor[plot_id],
                                 marker='o', s=marker.size, 
                                 alpha=marker.alpha
                                )
                
                if (label_marker == True):
                    label = marker.label
                else:
                    label = ' '
                    
                fig.add_label(marker.ra, marker.dec, '  ' + label, 
                              color=marker.color, fontsize=12, horizontalalignment='left')
                
            # label date
            try:
                date = self.date_dict[image_name].strip()
            except:
                date = 'unknown_date'
                
            date_label = 'Date : ' + date + '  JD : ' \
                         +  str(jd).strip()                    \
                         + '   Band : ' + band  
            fig.add_label(0.02, 0.95, date_label, relative=True, 
                         color=(0,1,1,1),
                         fontsize=9, horizontalalignment='left')
                    
            fig.axis_labels.hide()
            fig.save(outfig_name)    
            os.system('mv ' + outfig_name + ' ' + output_path_dict[band] )
            
        # pack output maps to be under the same directory
        for key in output_path_dict.keys():
            os.system('mv ' + output_path_dict[key] + ' ./' +  output_directory)   
            
        F_report.close()

In [6]:
'''
dmtau = apt_pipe(data_path = r"../3486034-dm tau",
                 markerfile   = './markers.txt', verbose=True)
dmtau.plot_preview(label_marker=False)
'''

'\ndmtau = apt_pipe(data_path = r"../3486034-dm tau",\n                 markerfile   = \'./markers.txt\', verbose=True)\ndmtau.plot_preview(label_marker=False)\n'

In [7]:
dmtau = apt_pipe(data_path = [
                              r"../3463766-dm tau",
                              r"../3486034-dm tau",
                              r"../3488473-dm tau",
                             ],
                 markerfile   = './markers.txt', verbose=True)
dmtau.plot_preview(label_marker=False)

Loading FITS image from : ../3463766-dm tau
Loading FITS image from : ../3486034-dm tau
Loading FITS image from : ../3488473-dm tau
##############################################################
Processing 117 images 

##############################################################
Warning, coordinate header of dm tau_3463766_V_002.fits does not exist.
Warning, coordinate header of dm tau_3463766_I_008.fits does not exist.
Warning, coordinate header of dm tau_3463766_V_000.fits does not exist.
Warning, coordinate header of dm tau_3463766_R_004.fits does not exist.
Warning, coordinate header of dm tau_3463766_R_005.fits does not exist.
Warning, coordinate header of dm tau_3463766_R_003.fits does not exist.
Warning, coordinate header of dm tau_3463766_I_006.fits does not exist.
Warning, coordinate header of dm tau_3463766_I_007.fits does not exist.
Warning, coordinate header of dm tau_3463766_V_001.fits does not exist.
Warning, coordinate header of dm tau_3488473_I_047.fits does not exist

##### Processing image 26   dm tau_3463766_V_028.fits
INFO: Auto-setting vmin to  3.702e+00 [aplpy.core]
INFO: Auto-setting vmax to  2.123e+02 [aplpy.core]
INFO: Auto-setting vmin to  2.828e+00 [aplpy.core]
INFO: Auto-setting vmax to  2.263e+02 [aplpy.core]
#### 26 ### 2458493.56462337 ### V_2458493.5646.png
##### saving image 26   dm tau_3463766_V_028.fits
##### Processing image 27   dm tau_3463766_V_010.fits
INFO: Auto-setting vmin to  5.491e+00 [aplpy.core]
INFO: Auto-setting vmax to  1.342e+02 [aplpy.core]
INFO: Auto-setting vmin to  1.584e+00 [aplpy.core]
INFO: Auto-setting vmax to  1.607e+02 [aplpy.core]
#### 27 ### 2458491.56406927 ### V_2458491.5641.png
##### saving image 27   dm tau_3463766_V_010.fits


/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


##### Processing image 28   dm tau_3463766_I_007.fits
##### Processing image 29   dm tau_3463766_V_029.fits
INFO: Auto-setting vmin to  4.129e+00 [aplpy.core]
INFO: Auto-setting vmax to  2.186e+02 [aplpy.core]
INFO: Auto-setting vmin to -5.429e+00 [aplpy.core]
INFO: Auto-setting vmax to  3.025e+02 [aplpy.core]
#### 29 ### 2458493.56540598 ### V_2458493.5654.png
##### saving image 29   dm tau_3463766_V_029.fits


/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


...

/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


##### Processing image 115   dm tau_3488473_I_041.fits
INFO: Auto-setting vmin to  9.780e+02 [aplpy.core]
INFO: Auto-setting vmax to  2.067e+03 [aplpy.core]
INFO: Auto-setting vmin to  1.005e+03 [aplpy.core]
INFO: Auto-setting vmax to  1.738e+03 [aplpy.core]
#### 115 ### 2458509.6171294 ### I_2458509.6171.png
##### saving image 115   dm tau_3488473_I_041.fits


/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


##### Processing image 116   dm tau_3488473_V_042.fits
INFO: Auto-setting vmin to  6.841e+00 [aplpy.core]
INFO: Auto-setting vmax to  2.067e+02 [aplpy.core]
INFO: Auto-setting vmin to  8.272e+00 [aplpy.core]
INFO: Auto-setting vmax to  1.798e+02 [aplpy.core]
#### 116 ### 2458510.61733225 ### V_2458510.6173.png
##### saving image 116   dm tau_3488473_V_042.fits


/home/hyliu/.conda/envs/terada2019/lib/python3.7/site-packages/aplpy/core.py:249: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self._figure = plt.figure(**kwargs)


In [ ]:
animals = ['cat', 'dog', 'rabbit']
d = ['test']
print(type(d) == list ) 
d = 'test'
print(type(d) == str ) 


animals.append('guinea pig')
print('Updated animals list: ', animals)
animals.extend(['guinea pig', 'moster'])
print('Updated animals list: ', animals)

In [ ]:
'''
# Directory for input image
data_path   = r"../3488473-dm tau"

# Defining output directories
output_path_dict = {}
output_path_dict['R'] = './R_maps'
output_path_dict['V'] = './V_maps'
output_path_dict['I'] = './I_maps'

# set if we want to do photometry for objects in the markers list
dophotometry     = True
aperature_r_pix  = 10.0 # pixels
if (dophotometry == True):
    print("Will output photometry. Please define the output filename if not yet.")
    photometry_file_dict = {}
    photometry_file_dict['R'] = './R_phot.txt'
    photometry_file_dict['V'] = './V_phot.txt'
    photometry_file_dict['I'] = './I_phot.txt'
    

# Output ASCII list for FITS images which do not possess coordinate headers
f = open("NO_COORD_HEADER_list.txt","w")    
    
markerfile = './markers.txt'
    
# Preparation ##############################################################
for key in output_path_dict.keys():
    os.system('rm -rf ' + output_path_dict[key] )
    os.system('mkdir '  + output_path_dict[key] )
    
if (dophotometry == True):
    for key in photometry_file_dict.keys():
        os.system('rm -rf ' + photometry_file_dict[key] )
        photo_file = open(photometry_file_dict[key],"a+")
        photo_file.write("# Data  target_name  JD  Counts")

num_markers = 0
try:
    marker_label = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=0, dtype=np.str)
    rah, ram, ras = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(1,2,3) )
    decd, decm, decs = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(4,5,6) )
    markerR, markerG, markerB = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=(7,8,9) )
    marker_alpha = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=10)
    marker_size  = np.loadtxt(markerfile, comments='#', skiprows=0, usecols=11)
    
    ra  = ( rah + ram / 60.0 + ras / 3600.0 ) * 15.0
    if (decd > 0 ):
        dec = decd + decm / 60.0 + decs / 3600.0
    else:
        dec = decd - decm / 60.0 - decs / 3600.0
    num_markers = np.size(ra)
    
except:
    print('No markers found')
    
images     = os.listdir( data_path )
num_images = len(images)
############################################################################
    

for i in range(0, num_images):
    image_name = images[i]
    
    info = image_name.strip('.fits').split('_')
    target_name    = info[0]
    directory_name = info[1]
    band           = info[2]
    epoch_idx      = info[3]
    
    # open FITS image
    hdulist = fits.open(data_path + '/' + image_name)
    try:
        crval1 = hdulist[0].header['crval1']
        crval2 = hdulist[0].header['crval2']
        date   = hdulist[0].header['date-obs']
        jd     = hdulist[0].header['jd']
    except:
        print('Error, coordinate header of ' + image_name + ' does not exist.')
        f.write( image_name + '\n' )
        continue

    # loading image
    if (dophotometry == True):
        try:
            image = hdulist[0].data
            w = wcs.WCS(hdulist[0].header)
        except:
            print("Error loading image " + directory_name + '_' + band + '_' + epoch_idx)
        
    
    fig = aplpy.FITSFigure(data_path + '/' + image_name)
    #fig.set_xaxis_coord_type('longitude')
    #fig.set_yaxis_coord_type('latitude')
    fig.axis_labels.hide()
    fig.show_grayscale(invert=False)
    
    # mark stars
    for j in range(0, num_markers):
        # plot markers in the png figure
        if (num_markers==1):
            x, y = ra, dec
            pixcrd2 = w.wcs_world2pix([ [x,y] ], 0)
            xpix = pixcrd2[0][0]
            ypix = pixcrd2[0][1]
            mcolor = (markerR, markerG, markerB)
            malpha = marker_alpha
            msize  = marker_size
            mlabel = str(marker_label)
        else:
            x, y = ra[j], dec[j]
            mcolor = (markerR[j], markerG[j], markerB[j])
            malpha = marker_alpha[j]
            msize = marker_size[j]
            mlabel = str(marker_label[j])
        
        fig.show_markers(
                         x, y, 
                         edgecolor=mcolor, 
                         # facecolor=facecolor[plot_id],
                         marker='o', s=msize, 
                         alpha=malpha
                        )
        fig.add_label(x, y, '  ' + mlabel, 
                      color=mcolor, fontsize=12, horizontalalignment='left')
        
        # optionally, do aperture photometry
        if (dophotometry == True):              
            # making photometry
            xpix_min = int(round(xpix - aperature_r_pix*3) )
            xpix_max = int(round(xpix + aperature_r_pix*3) )
            ypix_min = int(round(ypix - aperature_r_pix*3) )
            ypix_max = int(round(ypix + aperature_r_pix*3) )
            try:
                print(directory_name + '_' + band + '_' + epoch_idx)
                crop = image[ypix_min:ypix_max, xpix_min:xpix_max].astype(float)
                
                # background subtraction
                crop -= np.median(crop)
                
                # estimate pixel statistics
                bkg_sigma = mad_std(crop)
                
                # find stars
                daofind = DAOStarFinder(fwhm= aperature_r_pix/2.0, threshold=5.*bkg_sigma)
                sources = daofind(crop)

                
                mxcentroid_array = np.array( sources['xcentroid'] )
                mycentroid_array = np.array( sources['ycentroid'] )
                mpeak_array      = np.array( sources['peak']      )
                
                if ( len(mpeak_array) > 0 ):
                    # Very ugly code here due to unfamiliar with python.
                    # need to update. Baobab, 2019.Dec.25
                    index_array = range(0, len(mpeak_array) )
                    xindex = np.max(np.where( mpeak_array == np.max(mpeak_array), index_array, -1))
                    yindex = np.max(np.where( mpeak_array == np.max(mpeak_array), index_array, -1))
                    mxcentroid = mxcentroid_array[xindex]
                    mycentroid = mycentroid_array[yindex]
                    positions = np.transpose((mxcentroid, mycentroid))
                    apertures = CircularAperture(positions, aperature_r_pix)
                    phot_table = aperture_photometry(crop, apertures)
                    counts    = phot_table['aperture_sum'][0]

                    try:
                        photo_file = open(photometry_file_dict[band],"a+")
                        outtext     = directory_name + '_' + band + '_' + epoch_idx + '  ' + \
                                      str(marker_label) + '   ' + \
                                      str(jd).strip() + '   ' + \
                                      str(counts) + '   \n'
                        photo_file.write(outtext)
                        photo_file.close()
                    except:
                        print('Error opening output file. ' + directory_name + '_' + band + '_' + epoch_idx)
                    
                else:
                    print("No source found in" + directory_name + '_' + band + '_' + epoch_idx)
                    
                
            except:
                print("Error making photometry " + directory_name + '_' + band + '_' + epoch_idx)
                
            
        
    # label date
    date_label = 'Date : ' + date + '  JD : ' + str(jd).strip() + '   Band : ' + band  
    fig.add_label(0.02, 0.95, date_label, relative=True, 
                  color=(0,1,1,1),
                  fontsize=9, horizontalalignment='left')


    
    outfig_name = directory_name + '_' + band + '_' + epoch_idx + '.png'
    
    fig.save(outfig_name)    
    os.system('mv ' + outfig_name + ' ' + output_path_dict[band] )
    
    # close FITS image
    hdulist.close()
    
f.close()
'''

In [ ]:
test = []
test.append('I')
test.append('J')
print(test)

In [ ]:
round(1234.45678, 2)